# Web application

https://pypi.org/project/yfinance/

https://github.com/ranaroussi/yfinance

Short usage instructions for our needs:
`sym = yf.Ticker('YAHOOSYMBOL')` and data retrieval `sym.history(period, interval, auto_adjust=True)`

- period : valid values 1mo,5d,1d
- interval : valid values 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo

In [6]:
!pip install yfinance

  Running setup.py bdist_wheel for yfinance ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
  Running setup.py bdist_wheel for multitasking ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/37/fa/73/d492849e319038eb4d986f5152e4b19ffb1bc0639da84d2677
Successfully built yfinance multitasking
fastai 1.0.59 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import yfinance as yf

In [2]:
sym_ixic = yf.Ticker('^IXIC')

In [3]:
sym_ixic.history(period='5d', interval='1d')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-22,9413.61,9439.29,9375.13,9383.77,2449390000,0,0
2020-01-23,9377.72,9409.20,9334.13,9402.48,2460050000,0,0
2020-01-24,9446.21,9451.43,9273.23,9314.91,2611710000,0,0
2020-01-27,9092.46,9185.45,9088.04,9139.31,2583330000,0,0
2020-01-28,9201.82,9286.17,9182.33,9285.45,1325721370,0,0


In [4]:
sym_eurusd = yf.Ticker('EURUSD=X')

In [5]:
sym_eurusd.history(period='1d', interval='60m')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2020-01-28 00:00:00+00:00,1.1023,1.1025,1.1020,1.1023,0,0,0
2020-01-28 01:00:00+00:00,1.1023,1.1029,1.1023,1.1023,0,0,0
2020-01-28 02:00:00+00:00,1.1023,1.1027,1.1020,1.1024,0,0,0
2020-01-28 03:00:00+00:00,1.1024,1.1028,1.1023,1.1025,0,0,0
2020-01-28 04:00:00+00:00,1.1025,1.1028,1.1023,1.1025,0,0,0
2020-01-28 05:00:00+00:00,1.1028,1.1029,1.1024,1.1025,0,0,0
2020-01-28 06:00:00+00:00,1.1025,1.1029,1.1022,1.1023,0,0,0
2020-01-28 07:00:00+00:00,1.1027,1.1027,1.1017,1.1020,0,0,0
2020-01-28 08:00:00+00:00,1.1019,1.1025,1.1016,1.1022,0,0,0
